In [1]:
import pandas as pd
import numpy as np
import warnings
import sys
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from scipy import stats

warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = Path('Data')
dynamic_data_path = DATA_DIR / 'VED_DynamicData'
csv_files = []

veh_folders = sorted([f for f in dynamic_data_path.iterdir() if f.is_dir() and f.name.startswith('VehId_')])

for veh_folder in veh_folders:
    csv_in_folder = sorted(veh_folder.glob('*.csv'))
    csv_files.extend(csv_in_folder)

csv_files = [str(f) for f in csv_files]

print(f'Cantidad de csv: {len(csv_files)}')


Cantidad de csv: 32552


In [3]:
f = csv_files[0]
df0 = pd.read_csv(f, nrows=1000)
print('Dtypes:')
print(df0.dtypes)

Dtypes:
DayNum                            float64
VehId                               int64
Trip                                int64
Timestamp(ms)                       int64
Latitude[deg]                     float64
Longitude[deg]                    float64
Vehicle Speed[km/h]               float64
MAF[g/sec]                        float64
Engine RPM[RPM]                   float64
Absolute Load[%]                  float64
OAT[DegC]                         float64
Fuel Rate[L/hr]                   float64
Air Conditioning Power[kW]        float64
Air Conditioning Power[Watts]     float64
Heater Power[Watts]               float64
HV Battery Current[A]             float64
HV Battery SOC[%]                 float64
HV Battery Voltage[V]             float64
Short Term Fuel Trim Bank 1[%]    float64
Short Term Fuel Trim Bank 2[%]    float64
Long Term Fuel Trim Bank 1[%]     float64
Long Term Fuel Trim Bank 2[%]     float64
dtype: object


In [4]:
LABELS = {'DayNum', 'VehId', 'Trip', 'Timestamp(ms)', 'Latitude[deg]', 'Longitude[deg]'}

Y = {'Fuel Rate[L/hr]', 'HV Battery Current[A]'}

X = {
    'Engine RPM[RPM]',
    'Absolute Load[%]',
    'Vehicle Speed[km/h]',
    'MAF[g/sec]',
    'OAT[DegC]',
    'Air Conditioning Power[kW]',
    'Air Conditioning Power[Watts]',
    'Heater Power[Watts]',
    'HV Battery SOC[%]',
    'HV Battery Voltage[V]',
    'Short Term Fuel Trim Bank 1[%]',
    'Short Term Fuel Trim Bank 2[%]',
    'Long Term Fuel Trim Bank 1[%]',
    'Long Term Fuel Trim Bank 2[%]'
}

In [5]:
dir_path = DATA_DIR / 'VED_DynamicData'
dynamic_files = sorted(dir_path.glob('VehId_*/Trip_*.csv'))

if not dynamic_files:
    dynamic_files = sorted(dir_path.glob('*.csv'))

dynamic_files = [str(f) for f in dynamic_files]

# Static
static_data = pd.read_csv(DATA_DIR / 'VED_Static_Data_ICE&HEV&PHEV&EV.csv')
print(f"Loaded static data: {static_data.shape}")
print(f"Static data columns: {static_data.columns.tolist()}")

# Crear diccionario de tipos de vehículo para lookup rápido
veh_type_dict = dict(zip(static_data['VehId'], static_data['Vehicle Type']))
print(f"\nTipos de vehículos en dataset:")
print(static_data['Vehicle Type'].value_counts())

STATIC_COLUMNS_TO_ADD = [
    'Vehicle Type',
    'Vehicle Class',
    'Engine Configuration & Displacement',
    'Transmission',
    'Drive Wheels',
    'Generalized_Weight'
]

# Columnas excluidas del X para evitar data leakage
Y_ONLY_COLUMNS = {
    # Columnas para calcular consumo de combustión
    'Fuel Rate[L/hr]',
    'MAF[g/sec]',
    'Short Term Fuel Trim Bank 1[%]',
    'Short Term Fuel Trim Bank 2[%]',
    'Long Term Fuel Trim Bank 1[%]',
    'Long Term Fuel Trim Bank 2[%]',
    # Columnas para calcular consumo eléctrico
    'HV Battery Current[A]',
    'HV Battery Voltage[V]',
    'HV Battery SOC[%]',
}

X_FEATURES = X - Y_ONLY_COLUMNS

IDENTIFIER_COLUMNS = {'DayNum', 'VehId', 'Trip', 'Timestamp(ms)', 'Latitude[deg]', 'Longitude[deg]'}
X_ONLY_COLUMNS = X_FEATURES - IDENTIFIER_COLUMNS

# Constantes para calcular Fuel Rate desde MAF
AFR_GASOLINE = 14.7  # Air-Fuel Ratio estequiométrico para gasolina
FUEL_DENSITY = 0.75  # kg/L


def calculate_fuel_rate_from_maf(maf_g_per_sec):
    """
    Calcula Fuel Rate [L/hr] desde MAF [g/sec] usando la fórmula OBD-II.
    
    Fórmula: Fuel Rate = (MAF * 3600) / (AFR * ρ_fuel * 1000)
    
    Donde:
    - MAF: Mass Air Flow en g/sec
    - AFR: Air-Fuel Ratio (14.7 para gasolina)
    - ρ_fuel: Densidad del combustible (0.75 kg/L para gasolina)
    - 3600: segundos a horas
    - 1000: gramos a kg
    """
    return (maf_g_per_sec * 3600) / (AFR_GASOLINE * FUEL_DENSITY * 1000)


def calculate_metrics(series):
    series = series.fillna(0)
    metrics = {
        'mean': series.mean(),
        'median': series.median(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        'q25': series.quantile(0.25),
        'q75': series.quantile(0.75),
        'range': series.max() - series.min(),
    }
    return metrics

def apply_fourier_analysis(series, n_components=10):
    series = series.fillna(0).values
    
    if len(series) < 4:
        return {f'fft_magnitude_{i}': 0 for i in range(n_components)} | {f'fft_frequency_{i}': 0 for i in range(n_components)} | {'fft_energy': 0, 'fft_spectral_centroid': 0}
    
    fft_result = np.fft.fft(series)
    magnitude = np.abs(fft_result)
    frequencies = np.fft.fftfreq(len(series))
    
    n_samples = len(series)
    positive_freq_end = n_samples // 2
    
    mag_positive = magnitude[1:positive_freq_end]
    freq_positive = frequencies[1:positive_freq_end]
    
    if len(mag_positive) == 0:
        return {f'fft_magnitude_{i}': 0 for i in range(n_components)} | {f'fft_frequency_{i}': 0 for i in range(n_components)} | {'fft_energy': 0, 'fft_spectral_centroid': 0}
    
    sorted_indices = np.argsort(mag_positive)[::-1]
    n_comp = min(n_components, len(mag_positive))
    top_indices = sorted_indices[:n_comp]
    
    result = {}
    for i in range(n_comp):
        if i < len(top_indices):
            idx = top_indices[i]
            result[f'fft_magnitude_{i}'] = mag_positive[idx]
            result[f'fft_frequency_{i}'] = freq_positive[idx]
        else:
            result[f'fft_magnitude_{i}'] = 0
            result[f'fft_frequency_{i}'] = 0
    
    result['fft_energy'] = np.sum(magnitude ** 2)
    if np.sum(mag_positive) > 0:
        result['fft_spectral_centroid'] = np.sum(freq_positive * mag_positive) / np.sum(mag_positive)
    else:
        result['fft_spectral_centroid'] = 0
    
    return result

all_samples = []
skipped_short = 0
skipped_no_distance = 0

# Contadores por tipo de vehículo
stats_by_type = {
    'ICE': {'count': 0, 'combustion_valid': 0, 'electric_valid': 0},
    'HEV': {'count': 0, 'combustion_valid': 0, 'electric_valid': 0},
    'PHEV': {'count': 0, 'combustion_valid': 0, 'electric_valid': 0},
    'EV': {'count': 0, 'combustion_valid': 0, 'electric_valid': 0},
}

# Fuente de datos de combustión
fuel_from_maf_count = 0
fuel_from_direct_count = 0
fuel_no_data_count = 0

for file_path in dynamic_files:
    file_name = Path(file_path).stem
    df = pd.read_csv(file_path)
    
    if len(df) < 2:
        skipped_short += 1
        continue
    
    # Obtener VehId y tipo de vehículo
    veh_id = df['VehId'].iloc[0]
    veh_type = veh_type_dict.get(veh_id, 'Unknown')
    
    # Calcular intervalos de tiempo en horas
    time_intervals_hours = np.diff(df['Timestamp(ms)'].values) / (1000 * 3600)
    
    # Calcular distancia total (integración de velocidad * tiempo)
    speeds = df['Vehicle Speed[km/h]'].fillna(0).iloc[:-1].values
    distances_km = speeds * time_intervals_hours
    total_distance_km = np.sum(distances_km[distances_km > 0])
    
    if total_distance_km < 0.1:  # Mínimo 100 metros
        skipped_no_distance += 1
        continue
    
    # Actualizar contador
    if veh_type in stats_by_type:
        stats_by_type[veh_type]['count'] += 1
    
    # ============================================================
    # TARGET 1: CONSUMO DE COMBUSTIÓN (L/100km)
    # Solo para vehículos con motor de combustión: ICE, HEV, PHEV
    # Prioridad: 1. Fuel Rate directo, 2. Calcular desde MAF
    # ============================================================
    y_combustion = np.nan 
    
    if veh_type in ['ICE', 'HEV', 'PHEV']:
        fuel_rate_col = df['Fuel Rate[L/hr]'].fillna(0) if 'Fuel Rate[L/hr]' in df.columns else pd.Series([0]*len(df))
        maf_col = df['MAF[g/sec]'].fillna(0) if 'MAF[g/sec]' in df.columns else pd.Series([0]*len(df))
        
        # Verificar qué fuente usar
        if (fuel_rate_col > 0).any():
            fuel_rate = fuel_rate_col.iloc[:-1].values
            fuel_from_direct_count += 1
        elif (maf_col > 0).any():
            fuel_rate = calculate_fuel_rate_from_maf(maf_col.iloc[:-1].values)
            fuel_from_maf_count += 1
        else:
            fuel_rate = np.zeros(len(df) - 1)
            fuel_no_data_count += 1
        
        fuel_consumed_L = np.sum(fuel_rate * time_intervals_hours)
        
        if fuel_consumed_L > 0 and total_distance_km > 0:
            y_combustion = (fuel_consumed_L / total_distance_km) * 100  # L/100km
            if veh_type in stats_by_type:
                stats_by_type[veh_type]['combustion_valid'] += 1
        else:
            y_combustion = 0.0  # Sin consumo detectado
    
    # ============================================================
    # TARGET 2: CONSUMO ELÉCTRICO (kWh/km)
    # Solo para vehículos con batería HV: HEV, PHEV, EV
    # Fórmula: Potencia = V * I, Energía = Integrar Potencia * dt
    # ============================================================
    y_electric = np.nan 
    
    if veh_type in ['HEV', 'PHEV', 'EV']:
        voltage_col = df['HV Battery Voltage[V]'].fillna(0) if 'HV Battery Voltage[V]' in df.columns else pd.Series([0]*len(df))
        current_col = df['HV Battery Current[A]'].fillna(0) if 'HV Battery Current[A]' in df.columns else pd.Series([0]*len(df))
        
        voltage = voltage_col.iloc[:-1].values
        current = current_col.iloc[:-1].values
        
        # Verificar si hay datos de batería
        if np.any(voltage > 0):
            # Potencia instantánea en kW (V * A / 1000)
            power_kW = (voltage * current) / 1000
            
            # Energía total (kWh) - integramos potencia * tiempo
            # current positivo = descarga (consumo), negativo = carga (regeneración)
            energy_kWh = np.sum(power_kW * time_intervals_hours)
            
            # Usamos valor absoluto del consumo neto
            y_electric = abs(energy_kWh) / total_distance_km  # kWh/km
            
            if veh_type in stats_by_type:
                stats_by_type[veh_type]['electric_valid'] += 1
        else:
            y_electric = 0.0  # Sin datos de batería disponibles
    
    # Construccion de Reistro
    sample_row = {
        'filename': file_name,
        'VehId': veh_id,
        'DayNum': df['DayNum'].iloc[0],
        'Trip': df['Trip'].iloc[0],
        'total_distance_km': total_distance_km,
        'Y_consumption_combustion_L_per_100km': max(0, y_combustion) if not np.isnan(y_combustion) else np.nan,
        'Y_consumption_electric_kWh_per_km': max(0, y_electric) if not np.isnan(y_electric) else np.nan,
    }
    
    # Agregar datos estáticos
    veh_static = static_data[static_data['VehId'] == veh_id]
    if len(veh_static) > 0:
        for col in STATIC_COLUMNS_TO_ADD:
            if col in veh_static.columns:
                sample_row[col] = veh_static[col].iloc[0]
    else:
        for col in STATIC_COLUMNS_TO_ADD:
            sample_row[col] = np.nan
    
    # METRICS
    for col in sorted(X_ONLY_COLUMNS):
        if col in df.columns:
            metrics = calculate_metrics(df[col])
            for metric_name, metric_value in metrics.items():
                sample_row[f'{col}_{metric_name}'] = metric_value
    
    # FOURIER
    for col in sorted(X_ONLY_COLUMNS):
        if col in df.columns:
            fft_metrics = apply_fourier_analysis(df[col], n_components=5)
            for metric_name, metric_value in fft_metrics.items():
                sample_row[f'{col}_fft_{metric_name}'] = metric_value
    
    all_samples.append(sample_row)

df_combined = pd.DataFrame(all_samples)

# Reportes finales
print(f"RESUMEN DE PROCESAMIENTO")
print(f"Archivos procesados: {len(dynamic_files)}")
print(f"Viajes válidos: {len(all_samples)}")
print(f"Saltados (muy cortos): {skipped_short}")
print(f"Saltados (sin distancia): {skipped_no_distance}")

print(f"FUENTE DE DATOS DE COMBUSTIÓN")
print(f"Fuel Rate directo (columna original): {fuel_from_direct_count}")
print(f"Fuel Rate calculado desde MAF: {fuel_from_maf_count}")
print(f"Sin datos de combustible: {fuel_no_data_count}")

print(f"ESTADÍSTICAS POR TIPO DE VEHÍCULO")
for vtype, stats in stats_by_type.items():
    if stats['count'] > 0:
        comb_pct = stats['combustion_valid'] / stats['count'] * 100 if stats['count'] > 0 else 0
        elec_pct = stats['electric_valid'] / stats['count'] * 100 if stats['count'] > 0 else 0
        print(f"\n{vtype}:")
        print(f"- Total trips: {stats['count']}")
        print(f"- Combustión válida: {stats['combustion_valid']} ({comb_pct:.1f}%)")
        print(f"- Eléctrico válido: {stats['electric_valid']} ({elec_pct:.1f}%)")

# Estadísticas de los targets
print(f"ESTADÍSTICAS DE TARGETS (VALORES VÁLIDOS)")
y_comb = df_combined['Y_consumption_combustion_L_per_100km']
y_elec = df_combined['Y_consumption_electric_kWh_per_km']

print(f"\nConsumo Combustión (L/100km):")
print(f"- Registros aplicables: {y_comb.notna().sum()}")
print(f"- Con valor > 0: {(y_comb > 0).sum()}")
if (y_comb > 0).sum() > 0:
    print(f"- Media (>0): {y_comb[y_comb > 0].mean():.2f}")
    print(f"- Mediana (>0): {y_comb[y_comb > 0].median():.2f}")
    print(f"- Min (>0): {y_comb[y_comb > 0].min():.2f}")
    print(f"- Max: {y_comb.max():.2f}")

print(f"\nConsumo Eléctrico (kWh/km):")
print(f"- Registros aplicables: {y_elec.notna().sum()}")
print(f"- Con valor > 0: {(y_elec > 0).sum()}")
if (y_elec > 0).sum() > 0:
    print(f"- Media (>0): {y_elec[y_elec > 0].mean():.4f}")
    print(f"- Mediana (>0): {y_elec[y_elec > 0].median():.4f}")
    print(f"- Min (>0): {y_elec[y_elec > 0].min():.4f}")
    print(f"- Max: {y_elec.max():.4f}")

Loaded static data: (384, 7)
Static data columns: ['VehId', 'Vehicle Type', 'Vehicle Class', 'Engine Configuration & Displacement', 'Transmission', 'Drive Wheels', 'Generalized_Weight']

Tipos de vehículos en dataset:
Vehicle Type
ICE     264
HEV      93
PHEV     24
EV        3
Name: count, dtype: int64
RESUMEN DE PROCESAMIENTO
Archivos procesados: 32552
Viajes válidos: 32512
Saltados (muy cortos): 0
Saltados (sin distancia): 40
FUENTE DE DATOS DE COMBUSTIÓN
Fuel Rate directo (columna original): 377
Fuel Rate calculado desde MAF: 26256
Sin datos de combustible: 5376
ESTADÍSTICAS POR TIPO DE VEHÍCULO

ICE:
- Total trips: 18903
- Combustión válida: 14684 (77.7%)
- Eléctrico válido: 0 (0.0%)

HEV:
- Total trips: 9501
- Combustión válida: 9324 (98.1%)
- Eléctrico válido: 18 (0.2%)

PHEV:
- Total trips: 3605
- Combustión válida: 2625 (72.8%)
- Eléctrico válido: 3603 (99.9%)

EV:
- Total trips: 503
- Combustión válida: 0 (0.0%)
- Eléctrico válido: 503 (100.0%)
ESTADÍSTICAS DE TARGETS (VALORE

In [6]:
x_dir = DATA_DIR / 'X'
y_dir = DATA_DIR / 'Y'

x_dir.mkdir(parents=True, exist_ok=True)
y_dir.mkdir(parents=True, exist_ok=True)

Y_columns = [col for col in df_combined.columns if col.startswith('Y_')]
ID_columns = ['filename', 'VehId', 'DayNum', 'Trip']
EXTRA_columns = ['total_distance_km']  
STATIC_columns = STATIC_COLUMNS_TO_ADD
X_columns = [col for col in df_combined.columns 
             if col not in Y_columns 
             and col not in ID_columns 
             and col not in STATIC_columns
             and col not in EXTRA_columns]

X_metrics_columns = [col for col in X_columns if '_fft_' not in col]
X_fourier_columns = [col for col in X_columns if '_fft_' in col]

df_X_metrics = df_combined[ID_columns + STATIC_columns + X_metrics_columns].copy()
df_X_fourier = df_combined[ID_columns + STATIC_columns + X_fourier_columns].copy()
df_Y = df_combined[ID_columns + Y_columns].copy()

x_metrics_file = x_dir / 'X_metrics.csv'
df_X_metrics.to_csv(x_metrics_file, index=False)
x_fourier_file = x_dir / 'X_fourier.csv'
df_X_fourier.to_csv(x_fourier_file, index=False)
y_output_file = y_dir / 'Y.csv'
df_Y.to_csv(y_output_file, index=False)

print(f"Guardados:")
print(f"- {x_metrics_file}: {df_X_metrics.shape}")
print(f"- {x_fourier_file}: {df_X_fourier.shape}")
print(f"- {y_output_file}: {df_Y.shape}")

Guardados:
- Data\X\X_metrics.csv: (32512, 66)
- Data\X\X_fourier.csv: (32512, 94)
- Data\Y\Y.csv: (32512, 6)


In [7]:
df_X_metrics = pd.read_csv(DATA_DIR / 'X' / 'X_metrics.csv')
df_X_fourier = pd.read_csv(DATA_DIR / 'X' / 'X_fourier.csv')

categorical_cols = ['Vehicle Type', 'Vehicle Class', 'Transmission', 'Drive Wheels', 'Engine Configuration & Displacement']

for col in categorical_cols:
    if col in df_X_metrics.columns:
        print(f"\n{col}:")
        unique_vals = df_X_metrics[col].unique()
        null_count = df_X_metrics[col].isna().sum()
        null_pct = (null_count / len(df_X_metrics)) * 100
        
        print(f"Valores únicos: {len(unique_vals)}")
        print(f"Valores faltantes: {null_count} ({null_pct:.2f}%)")
        
        value_counts = df_X_metrics[col].value_counts(dropna=False)
        for val, count in value_counts.items():
            pct = (count / len(df_X_metrics)) * 100
            print(f"    {val}: {count} ({pct:.2f}%)")

vt_onehot = pd.get_dummies(df_X_metrics['Vehicle Type'], prefix='VehicleType', dummy_na=False)
vt_onehot = vt_onehot.loc[:, ~vt_onehot.columns.str.contains('NO DATA', case=False, na=False)]

dw_onehot = pd.get_dummies(df_X_metrics['Drive Wheels'], prefix='DriveWheels', dummy_na=False)
dw_onehot = dw_onehot.loc[:, ~dw_onehot.columns.str.contains('NO DATA', case=False, na=False)]

le_vc = LabelEncoder()
df_X_metrics['Vehicle_Class_encoded'] = le_vc.fit_transform(df_X_metrics['Vehicle Class'].fillna('MISSING'))
print(f"Mapeo: {dict(zip(le_vc.classes_, le_vc.transform(le_vc.classes_)))}")

le_trans = LabelEncoder()
df_X_metrics['Transmission_encoded'] = le_trans.fit_transform(df_X_metrics['Transmission'].fillna('MISSING'))

ecd_unique = df_X_metrics['Engine Configuration & Displacement'].unique()
ecd_count = df_X_metrics['Engine Configuration & Displacement'].value_counts(dropna=False)

X_metrics_processed = df_X_metrics.copy()
X_fourier_processed = df_X_fourier.copy()

X_metrics_processed = pd.concat([X_metrics_processed, vt_onehot, dw_onehot], axis=1)
X_fourier_processed = pd.concat([X_fourier_processed, vt_onehot, dw_onehot], axis=1)

X_metrics_processed['Vehicle_Class_encoded'] = le_vc.fit_transform(X_metrics_processed['Vehicle Class'].fillna('MISSING'))
X_metrics_processed['Transmission_encoded'] = le_trans.fit_transform(X_metrics_processed['Transmission'].fillna('MISSING'))

X_fourier_processed['Vehicle_Class_encoded'] = le_vc.transform(X_fourier_processed['Vehicle Class'].fillna('MISSING'))
X_fourier_processed['Transmission_encoded'] = le_trans.transform(X_fourier_processed['Transmission'].fillna('MISSING'))

X_metrics_processed.to_csv(DATA_DIR / 'X' / 'X_metrics_processed.csv', index=False)
X_fourier_processed.to_csv(DATA_DIR / 'X' / 'X_fourier_processed.csv', index=False)


Vehicle Type:
Valores únicos: 4
Valores faltantes: 0 (0.00%)
    ICE: 18903 (58.14%)
    HEV: 9501 (29.22%)
    PHEV: 3605 (11.09%)
    EV: 503 (1.55%)

Vehicle Class:
Valores únicos: 3
Valores faltantes: 0 (0.00%)
    NO DATA: 28161 (86.62%)
    Car: 4349 (13.38%)
    SUV: 2 (0.01%)

Transmission:
Valores únicos: 19
Valores faltantes: 180 (0.55%)
    NO DATA: 23342 (71.80%)
    CVT: 3463 (10.65%)
    5-SP AUTOMATIC: 1947 (5.99%)
    AUTOMATIC/CVT: 1138 (3.50%)
    4-SP AUTOMATIC: 588 (1.81%)
    AUTOMATIC: 385 (1.18%)
    4-SP Automatic: 361 (1.11%)
    5-SP MANUAL: 259 (0.80%)
    nan: 180 (0.55%)
    6-SP ECT AUTOMATIC: 164 (0.50%)
    FULL TIME 4WD MANUAL: 152 (0.47%)
    5-SP Automatic: 137 (0.42%)
    6-SP AUTOMATIC: 129 (0.40%)
    5-SP AWD MANUAL: 85 (0.26%)
    9-SP Automatic: 68 (0.21%)
    5-SP ECT AUTOMATIC: 51 (0.16%)
    FULL TIME 4WD AUTOMATIC: 49 (0.15%)
    6-SP AWD MANUAL: 13 (0.04%)
    Automatic: 1 (0.00%)

Drive Wheels:
Valores únicos: 2
Valores faltantes: 0 (0.00